In [17]:
!./build.sh

[NbConvertApp] Converting notebook nb-dev-create-job.ipynb to script
[NbConvertApp] Writing 8585 bytes to nb-dev-create-job.py
creating pyfile: create_job.py


In [18]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports create_job.py

=== Sorting with ISORT ===
Fixing /Users/ALAMSHC/PycharmProjects/AwsInternalBusiness/serverside/lambdas/create_job/create_job.py
=== Formatting with AUTOPEP8 ===
=== Testing with Flake8 ===
=== Please deleted the sections with marked ===> before further deployment ===
=== NOW PLEASE REMOVE THE TEST CODE FROM THE GENERATED FILES ===
=== Testing Removing Unused Import and Varialbe ===


In [19]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)
import json

# Test Lambda

In [20]:
from create_job import handler

In [21]:
# Testing the Result
test_event_json = {
    "sk": os.environ["TEMP_PROJECT_NAME"], 
    "requested_by": "pluto"
}

# process_request(test_event_json)
test_event = {"body": json.dumps(test_event_json)}
response = handler(event=test_event, context=_)

In [22]:
assert response['statusCode'] == 200 

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [23]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/create-job-step.zip"
FUNCTION_NAME = "quack-tsln-create-job-step"
ENV_DYNAMODB = os.environ["DB_NAME"]

ENV_VARIABLES = {
    'Variables': {
        'BUCKET': BUCKET, 
        'DB_NAME': ENV_DYNAMODB, 
        "LAMBDA_RUNTIME_ENV": "cloud",
        "SFN_WORKFLOW_ARN": os.environ["SFN_WORKFLOW_ARN"]
    }
}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [24]:
build_package(PACKAGE)

2022-12-12 17:55:48 [INFO] deploykit - orm.py file added
2022-12-12 17:55:48 [INFO] deploykit - datamodel.py file added
2022-12-12 17:55:48 [INFO] deploykit - __init__.py file added
2022-12-12 17:55:48 [INFO] deploykit - logger.py file added
2022-12-12 17:55:48 [INFO] deploykit - utils.py file added
2022-12-12 17:55:48 [INFO] deploykit - exceptions.py file added
2022-12-12 17:55:48 [INFO] deploykit - domainmodel.py file added
2022-12-12 17:55:48 [INFO] deploykit - create_job.py file added
2022-12-12 17:55:48 [INFO] deploykit - Pyfile zipped!
2022-12-12 17:55:48 [INFO] deploykit - Package Zipped


In [25]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

2022-12-12 17:55:50 [INFO] deploykit - Zipfile Uploaded into s3://quack-translation-buddy/lambdas/create-job-step.zip


In [26]:
DESC = "Create translation from a step function"
MAIN_PYFILE = "create_job"

In [27]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES,
        code_info=CODE_INFO,
        memory_size=128
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

An error occurred (ResourceConflictException) when calling the CreateFunction operation: Function already exist: quack-tsln-create-job-step


2022-12-12 17:55:55 [INFO] deploykit - Lambda Code Updated


In [16]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)

2022-12-12 16:00:56 [INFO] deploykit - Environ Variables Updated


In [ ]:
# Create Query Parameter
# "&".join([f"{key}={value}" for key, value in test_event["queryStringParameters"].items()])